# <center>Data Analysis & Transformation </center>

The purpose of this notebook is to analyze data and prepare it by joining multiple sources, picking up the right features, cleaning them and finally saving as a ready-to-go Azure ML Dataset. 

I will use 5 data sources from the provided link:
- <b>ontime_reporting (from both years - combined) </b>
- <b>airport_weather (from both years - combined)</b> 
- <b>B43_aircraft_inventory </b> 
- <b>airport_list </b> 
- <b>airport_coordinates </b> 

Based on my reaserch, there are the most common reasons for flight delays.

<img src="https://www.claimcompass.eu/blog/content/images/2019/09/Why-is-My-Flight-Delayed-The-Main-Reasons.png">

Not all of them can be describe with available data. Here are the features I decided to use in that experiment.
- ontime_reporting
    - distance
    - planned departure time
    - day of week
<br><br>
- weather
    - PRCP: Inches of precipitation for day (rain)
    - SNOW: Inches of snowfall for day
    - SNWD: Inches of snow on ground for day
    - TMAX: Max temperature for day
    - AWND: Max wind speed for day
<br><br>    
- B43_Aircraft_inventory
    - number_of_seats
<br><br>
- airport_coordinates
    - latitiude
    - longitiude

All files have been converted to Datasets (manually - using Azure ML Studio), so now let's prepare the final dataset the will be used to train the model.

In [1]:
#imports

import azureml.core
from azureml.core import Workspace, Dataset, Datastore
import pandas as pd
import numpy as np

In [2]:
# Load the workspace from the saved config file

ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.44.0 to work with avanade-airline-delays


In [3]:
# Load all datasets

print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

Datasets:
	 airlines_processed_df_final version 1
	 airlines_processed_dataset version 1
	 airport_coordinates version 1
	 aircraft_inventory version 1
	 airport_list version 1
	 airport_weather_2020 version 1
	 airport_weather_2019 version 1
	 ontime_reporting_2020 version 1
	 ontime_reporting_2019 version 1


In [4]:
aircraft_inventory = Dataset.get_by_name(ws, name='aircraft_inventory')
df_aircraft_inventory = aircraft_inventory.to_pandas_dataframe()

airport_list = Dataset.get_by_name(ws, name='airport_list')
df_airport_list = airport_list.to_pandas_dataframe()

airport_weather_2020 = Dataset.get_by_name(ws, name='airport_weather_2020')
df_airport_weather_2020 = airport_weather_2020.to_pandas_dataframe()

airport_weather_2019 = Dataset.get_by_name(ws, name='airport_weather_2019')
df_airport_weather_2019 = airport_weather_2019.to_pandas_dataframe()

ontime_reporting_2020 = Dataset.get_by_name(ws, name='ontime_reporting_2020')
df_ontime_reporting_2020 = ontime_reporting_2020.to_pandas_dataframe()

ontime_reporting_2019 = Dataset.get_by_name(ws, name='ontime_reporting_2019')
df_ontime_reporting_2019 = ontime_reporting_2019.to_pandas_dataframe()

airport_coordinates = Dataset.get_by_name(ws, name='airport_coordinates')
df_airport_coordinates = airport_coordinates.to_pandas_dataframe()

Let's start by selecting the right features for each dataset.

In [7]:
# weather df // 2 DFs will be concatenated firts as all rows are makred with proper date
df_airport_weather = pd.concat([df_airport_weather_2019, df_airport_weather_2020])
df_airport_weather = df_airport_weather[['NAME', 'DATE', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'AWND']]

# airport_list
df_airport_list = df_airport_list[['ORIGIN_AIRPORT_ID', 'NAME']]

# ontime_reporting
df_ontime_reporting_2019 = df_ontime_reporting_2019[['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'TAIL_NUM', 'ORIGIN_AIRPORT_ID',
                                                    'DISTANCE','CRS_DEP_TIME' ,'DEP_DEL15']]

df_ontime_reporting_2020 = df_ontime_reporting_2020[['MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'TAIL_NUM', 'ORIGIN_AIRPORT_ID',
                                                    'DISTANCE','CRS_DEP_TIME' ,'DEP_DEL15']]

Now, let's prepare the onetime_reporting dfs, co they could be concatenated and merged with other datasets.

In [8]:
# add 'YEAR' columns and based on that 'DATE' columns
df_ontime_reporting_2019['YEAR'] = 2019
df_ontime_reporting_2020['YEAR'] = 2020

#2019
df_ontime_reporting_2019.rename(columns={'DAY_OF_MONTH': 'DAY'}, inplace = True)
df_ontime_reporting_2019['DATE'] = pd.to_datetime(df_ontime_reporting_2019[['YEAR', 'MONTH', 'DAY']])

#2020
df_ontime_reporting_2020.rename(columns={'DAY_OF_MONTH': 'DAY'}, inplace = True)
df_ontime_reporting_2020['DATE'] = pd.to_datetime(df_ontime_reporting_2020[['YEAR', 'MONTH', 'DAY']])

# drop columns 'YEAR', 'MONTH' and 'DAY', as they are not needed anymore 
df_ontime_reporting_2019.drop(['YEAR','MONTH', 'DAY'], axis = 1, inplace = True)
df_ontime_reporting_2020.drop(['YEAR','MONTH', 'DAY'], axis = 1, inplace = True)

Let's merge the two ontime_reporting dfs and join them with the rest of datasets.

In [9]:
df_ontime_reporting = pd.concat([df_ontime_reporting_2019, df_ontime_reporting_2020])

#join df_ontime_reporting with df_airport_list  in order to get airport names
df_draft1 = pd.merge(df_ontime_reporting, df_airport_list, how = 'left', on = 'ORIGIN_AIRPORT_ID')

#join above df with df_airport_weather
df_draft2 = pd.merge(df_draft1, df_airport_weather, how = 'left', on = ['NAME', 'DATE'])

#join above df with df_airport_coordinates

#remove duplicates from df_airport_coordinates
df_airport_coordinates.drop_duplicates(subset=['ORIGIN_AIRPORT_ID'], inplace = True)

df_draft3 = pd.merge(df_draft2, df_airport_coordinates, how = 'left', on = 'ORIGIN_AIRPORT_ID')

#join above df with df_aircraft_inventory to get the number of seats

#remove duplicates from df_aircraft_inventory
df_aircraft_inventory.drop_duplicates(subset=['TAIL_NUM'], inplace = True)

df_final = pd.merge(df_draft3, df_aircraft_inventory, how = 'left', on = 'TAIL_NUM')

In [10]:
df_final_copy = df_final.copy()
#df_final = df_final_copy

Let's display the final DF and process it further.

In [11]:
df_final.describe()

,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,DISTANCE,CRS_DEP_TIME,DEP_DEL15,PRCP,SNOW,SNWD,TMAX,AWND,LATITUDE,LONGITUDE,NUMBER_OF_SEATS
count,9.304362e+06,9.304362e+06,9.304362e+06,9.304362e+06,9.051745e+06,7.191868e+06,4.846610e+06,4.705170e+06,7.192458e+06,7.193058e+06,9.304362e+06,9.304362e+06,8.825005e+06
mean,3.946717e+00,1.265165e+04,8.013792e+02,1.331048e+03,1.797580e-01,1.025322e-01,5.869697e-02,1.493159e-01,6.910917e+01,8.416456e+00,3.672162e+01,-9.414409e+01,1.289433e+02
std,1.986160e+00,1.524566e+03,5.918103e+02,4.935397e+02,3.839858e-01,3.034343e-01,4.325924e-01,9.125906e-01,1.857393e+01,3.650529e+00,5.866546e+00,1.823289e+01,4.911599e+01
min,1.000000e+00,1.013500e+04,3.100000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.300000e+01,0.000000e+00,-1.433444e+01,-1.766447e+02,0.000000e+00
25%,2.000000e+00,1.129200e+04,3.690000e+02,9.130000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.600000e+01,5.820000e+00,3.343611e+01,-1.048797e+02,7.600000e+01
50%,4.000000e+00,1.288900e+04,6.410000e+02,1.324000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.100000e+01,7.830000e+00,3.736278e+01,-8.790611e+01,1.430000e+02
75%,6.000000e+00,1.393100e+04,1.035000e+03,1.739000e+03,0.000000e+00,3.000000e-02,0.000000e+00,0.000000e+00,8.400000e+01,1.051000e+01,4.077944e+01,-8.093583e+01,1.690000e+02
max,7.000000e+00,1.686900e+04,5.095000e+03,2.359000e+03,1.000000e+00,7.130000e+00,9.900000e+00,2.520000e+01,1.150000e+02,3.288000e+01,7.128556e+01,1.457303e+02,3.370000e+02


In [12]:
df_final.head(3)

,DAY_OF_WEEK,TAIL_NUM,ORIGIN_AIRPORT_ID,DISTANCE,CRS_DEP_TIME,DEP_DEL15,DATE,NAME,PRCP,SNOW,SNWD,TMAX,AWND,LATITUDE,LONGITUDE,NUMBER_OF_SEATS
0,7,N8694A,10397,83.0,1645,0.0,2019-01-06,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,0.0,0.0,0.0,69.0,6.49,33.640833,-84.427222,50.0
1,1,N8970D,10397,83.0,1645,0.0,2019-01-07,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,0.0,0.0,0.0,69.0,5.14,33.640833,-84.427222,50.0
2,2,N820AY,10397,83.0,1645,0.0,2019-01-08,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,0.0,0.0,0.0,65.0,8.05,33.640833,-84.427222,50.0


In [13]:
# remove not needed columns
df_final.drop(columns = ['NAME', 'DATE', 'ORIGIN_AIRPORT_ID', 'TAIL_NUM'], inplace = True)

#convert column 'DEP_DEL15' to int and change its name to 'TARGET'. Remove rows that doesn't equal 1 or 0
df_final = df_final[(df_final['DEP_DEL15'] == 1.0) | (df_final['DEP_DEL15'] == 0.0)]
df_final = df_final.astype({'DEP_DEL15': 'int32'})
df_final.rename(columns = {'DEP_DEL15': 'TARGET'}, inplace = True)

In [14]:
#replace all the nulls with column's median
df_final['PRCP'].fillna(df_final['PRCP'].median(), inplace = True)
df_final['SNOW'].fillna(df_final['SNOW'].median(), inplace = True)
df_final['SNWD'].fillna(df_final['SNWD'].median(), inplace = True)
df_final['TMAX'].fillna(df_final['TMAX'].median(), inplace = True)
df_final['AWND'].fillna(df_final['AWND'].median(), inplace = True)
df_final['NUMBER_OF_SEATS'].fillna(df_final['NUMBER_OF_SEATS'].median(), inplace = True)

In [15]:
#create a column that categorizes the values into four labels: night, morning, afternoon, evening.
#Later, I will use one-hot-encoding method to split the values into separate columns.
#original column 'CRS_DEP_TIME' will be removed

def dep_time(x):
    
    if x < 600:
        return 'NIGHT'
    elif x >= 600 and x < 1200:
        return 'MORNING'
    elif x >= 1200 and x < 1800:
        return 'AFTERNOON'
    else:
        return 'EVENING'
        
#apply function
df_final['DEP_TIME'] = df_final['CRS_DEP_TIME'].apply(lambda x: dep_time(x))

#one-hot encoding
df_onehot = pd.get_dummies(df_final['DEP_TIME'], prefix  = "DEP_TIME")

#merge with df_final
df_final = df_final.join(df_onehot)

#remove not needed columns
df_final.drop(columns = ['CRS_DEP_TIME', 'DEP_TIME'], inplace = True)

I will also aply the one-hot encoding method to column 'DAY_OF_WEEK'

In [16]:
df_onehot_days = pd.get_dummies(df_final['DAY_OF_WEEK'], prefix  = "DAY_OF_WEEK")

#merge with df_final
df_final = df_final.join(df_onehot_days)

#remove oryginal column
df_final.drop(columns = ['DAY_OF_WEEK'], inplace = True)

I want to also deal with potential ouliers. To do this I will apply a <b>Mean and Standard Deviation Method</b>
<br><br>
> For this outlier detection method, the mean and standard deviation of the residuals are calculated and compared. If a value is a certain number of standard deviations away from the mean, that data point is identified as an outlier. The specified number of standard deviations is called the threshold. The default value is 3.

https://docs.oracle.com/cd/E40248_01/epm.1112/cb_statistical/frameset.htm?ch07s02s10s01.html

### In order to prepare dataset 'airlines_processed_df_final_2' this step has been skiped.

In [17]:
# outliers = ['DISTANCE','PRCP', 'SNOW', 'SNWD', 'TMAX', 'AWND', 'NUMBER_OF_SEATS']

# for col in outliers:
#     print('Column ', col, ' is being processed...')
#     print('Max value now: ', df_final[col].max())
#     print('Min value now: ', df_final[col].min())
#     print('Count of rows: ', df_final[col].count(), '\n')
    
#     df_final = df_final[np.abs(df_final[col]-df_final[col].mean()) <= (3*df_final[col].std())]
    
#     print('Column ', col, ' processed')
#     print('Max value now: ', df_final[col].max())
#     print('Min value now: ', df_final[col].min())
#     print('Count of rows now: ', df_final[col].count(), '\n')
#     print('#########################################\n')

Let's check if there are any nulls left.

In [19]:
df_final.isna().sum()

In [20]:
df_final.describe()

Finally, I will use SKLearn's MaxAbsScaler to standarize all the values and convert 'TARGET' column to int type.

In [21]:
from sklearn.preprocessing import MaxAbsScaler

transformer = MaxAbsScaler().fit(df_final)
df_transformed = transformer.transform(df_final)

columns_names = df_final.columns

#final, processed DataFrame
df_ready = pd.DataFrame(data = df_transformed, columns = columns_names)
df_ready = df_ready = df_ready.astype({'TARGET': 'int32'})

In [25]:
df_ready.head()

,DISTANCE,TARGET,PRCP,SNOW,SNWD,TMAX,AWND,LATITUDE,LONGITUDE,NUMBER_OF_SEATS,...,DEP_TIME_EVENING,DEP_TIME_MORNING,DEP_TIME_NIGHT,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6,DAY_OF_WEEK_7
0,0.01629,0,0.0,0.0,0.0,0.600000,0.197384,0.471917,-0.477949,0.148368,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.01629,0,0.0,0.0,0.0,0.600000,0.156326,0.471917,-0.477949,0.148368,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.01629,0,0.0,0.0,0.0,0.565217,0.244830,0.471917,-0.477949,0.148368,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.01629,0,0.0,0.0,0.0,0.486957,0.530718,0.471917,-0.477949,0.148368,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.01629,0,0.0,0.0,0.0,0.391304,0.401460,0.471917,-0.477949,0.148368,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Above DataFrame can now be registered as an Azure ML Dataset and used to train the models.

In [23]:
#get my default datastore
default_ds = ws.get_default_datastore()

#register dataset in Azure ML
#dataset = Dataset.Tabular.register_pandas_dataframe(df_ready, default_ds, "airlines_processed_dataset", show_progress=True)

In [26]:
#save df_final as a dataset
dataset = Dataset.Tabular.register_pandas_dataframe(df_ready, default_ds, "airlines_processed_df_final_2", show_progress=True)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/a5e343cc-ba07-4955-9f18-faf7a7fe7474/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


Next step in the <b>Auto ML Notebook</b>